In [37]:
#Import basic python packages for data analysis and plotting
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import matplotlib.lines as mlines
import pylab as plot
import matplotlib
import random
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.axes_grid1 import make_axes_locatable
import math
import time
import os

import torch

## Pytorch practice

In [50]:
data = [[1, 2],[3, 4]]
x_data = torch.tensor(data)
x_data

shape = (2,3,)
rand_tensor = torch.rand(shape)
rand_tensor

tensor = torch.ones(4, 4)
tensor.shape

t1 = torch.cat([tensor, tensor, tensor], dim=-2)

t = torch.ones(5)
t

n = t.numpy()
print(f"n: {n}")

t.add_(1)
print(f"t: {t}")
print(f"n: {n}")

n: [1. 1. 1. 1. 1.]
t: tensor([2., 2., 2., 2., 2.])
n: [2. 2. 2. 2. 2.]


In [56]:
t.sum()

tensor(10.)

## (A) Dataset prep

In [38]:
# merge all datasets to create master_df

root = os.path.dirname(os.path.dirname(os.getcwd()))

df_aq = pd.read_csv(root + "/data/cleaned/air_quality_NO2.csv", index_col=0)[['value','latitude', 'longitude']]
df_met = pd.read_csv(root + "/data/cleaned/nO2_met.csv", index_col=0)
df_fac = pd.read_csv(root + "/data/cleaned/no2_fac_data.csv", index_col=0)
# df_fac.drop(df_fac.columns[df_fac.columns.str.contains('_emsdist')], axis=1, inplace=True)
df_traffic = pd.read_csv(root + "/data/cleaned/intersection_final.csv", index_col=0)

df_m1 = df_aq.merge(df_met, on = ['latitude', 'longitude'], how = 'inner')
df_m2 = df_m1.merge(df_fac, on = ['latitude', 'longitude'], how = 'inner')
df_merged = df_m2.merge(df_traffic, on = ['latitude', 'longitude'], how = 'inner')
df_merged.drop(columns = ['latitude', 'longitude'], inplace=True)

X = df_merged.drop("value",1) 
y = df_merged["value"]

/var/folders/f8/zhyxbtd52514w0pg6p51k9q40000gn/T/ipykernel_41119/3021703729.py:16: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = df_merged.drop("value",1)


In [39]:
# function to scale and transform input data
def get_data(X, y):
    X = X.values
    y = y.values
    # scaling the data
    feature_scaler = StandardScaler()
    X = feature_scaler.fit_transform(X)
    return X, y

# acquiring transformed data
X_arr, y_arr = get_data(X, y)

# splitting into test and train
X_train, X_test, y_train, y_test = train_test_split(X_arr, y_arr, test_size=0.3)
cols = np.array(X.columns)

## (B) Modeling

In [45]:
# dataset class for feeding in data
class AirQualityDataset(torch.utils.data.Dataset):

  def __init__(self, X_arr, y_arr):
    self.x_data = torch.tensor(X_arr, \
      dtype=torch.float32)
    self.y_data = torch.tensor(y_arr, \
      dtype=torch.float32)

  def __len__(self):
    return len(self.x_data)

  def __getitem__(self, idx):
    preds = self.x_data[idx,:]  # or just [idx]
    conc = self.y_data[idx] 
    return (preds, conc)       # tuple of matrices

# prepping data for training
batch_size = 64
train_ds = AirQualityDataset(X_train, y_train)
test_ds = AirQualityDataset(X_test, y_test)
train_ldr = torch.utils.data.DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_ldr = torch.utils.data.DataLoader(test_ds, batch_size=batch_size, shuffle=True)

In [58]:
# network architecture
class NeuralNetwork(torch.nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.linear_relu_stack = torch.nn.Sequential(
            torch.nn.Linear(82, 30),
            torch.nn.ReLU(),
            torch.nn.Linear(30, 30),
            torch.nn.ReLU(),
            torch.nn.Linear(30, 1),
        )

    def forward(self, x):
        pred_conc = self.linear_relu_stack(x)
        return pred_conc

In [59]:
# creating model instance
model = NeuralNetwork()
print(model)

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")
    
# initialising hyperparameters
learning_rate = 1e-3
epochs = 5

# initializing the loss function
loss_fn = torch.nn.MSELoss()

# initializing the optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=82, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=30, bias=True)
    (3): ReLU()
    (4): Linear(in_features=30, out_features=1, bias=True)
  )
)
Layer: linear_relu_stack.0.weight | Size: torch.Size([30, 82]) | Values : tensor([[ 0.0711,  0.0862, -0.0837, -0.0313,  0.0060,  0.0882,  0.0062,  0.1098,
          0.0945, -0.0447,  0.0521, -0.0102,  0.0528,  0.0277,  0.0777, -0.0299,
          0.0023,  0.0891, -0.1081,  0.0714,  0.1047, -0.0721,  0.0096, -0.0374,
         -0.0036, -0.0153,  0.0005,  0.0795,  0.0665, -0.0435,  0.0620, -0.0652,
          0.0344, -0.1038, -0.0098,  0.0268, -0.0803,  0.0155,  0.0569,  0.0642,
         -0.1034,  0.0868,  0.0784, -0.0442, -0.0621, -0.0739, -0.0211, -0.1031,
          0.0683, -0.0965,  0.0502, -0.0412, -0.0050, -0.0797, -0.1089, -0.0950,
          0.0564,  0.0956,  0.0557, -0.1011,  0.0364, -0.0131, -0.0120, -0.0118,
          0

In [60]:
# defining train and test loops

def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: Avg loss: {test_loss:>8f} \n")

In [61]:
# executing training and testing
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_ldr, model, loss_fn, optimizer)
    test_loop(test_ldr, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 0.021040  [    0/ 7919]
loss: 0.018013  [ 6400/ 7919]
Test Error: Avg loss: 0.007193 

Epoch 2
-------------------------------
loss: 0.004702  [    0/ 7919]
loss: 0.006121  [ 6400/ 7919]
Test Error: Avg loss: 0.005303 

Epoch 3
-------------------------------
loss: 0.003678  [    0/ 7919]
loss: 0.003023  [ 6400/ 7919]
Test Error: Avg loss: 0.004612 

Epoch 4
-------------------------------
loss: 0.003738  [    0/ 7919]
loss: 0.004570  [ 6400/ 7919]
Test Error: Avg loss: 0.003447 

Epoch 5
-------------------------------
loss: 0.002368  [    0/ 7919]
loss: 0.003972  [ 6400/ 7919]
Test Error: Avg loss: 0.002944 

Done!
